<h1>Building and Training Siamese Neural Network</h1>

In [1]:
# Local package imports
from src.data_loader import DataSplitter
from src.data_loader.FaceRecognitionDataset import FaceRecognitionDataset
from src.model.FaceNet import SiameseNetwork
from src.trainer.FaceNetTrainer import SiameseNetworkTrainer
from src.utils import mytensorboard
import src.utils.utils_images as img_util

# Pytorch imports 
import torch
import torchvision
from torchsummary import summary

# General imports
import time

<h3>General/Global Variables</h3>

In [2]:
# Paths 
dataset_dir ="./src/data/celeba_dataset/images/"
labels_path="./src/data/celeba_dataset/labels.txt"

# Hyperparameters
batch_size = 256
val_ratio = 0.5
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

<h3>Data loading and preprocessing</h3>

In [3]:
print("--------------------------------------------------------------------------")
print("Create dataset...")
dataset = FaceRecognitionDataset(dataset_dir=dataset_dir, labels_path=labels_path)
print(f"Created dataset! Number of samples: {len(dataset)}")
print("--------------------------------------------------------------------------")
print("Split dataset...")
train_dataset, val_dataset = DataSplitter.split_train_test(dataset=dataset, val_ratio=val_ratio)
print(f"Splitted dataset! Number of training samples: {len(train_dataset)}, number of validation samples: {len(val_dataset)}")
print("--------------------------------------------------------------------------")
print("Create data loaders...")
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           num_workers=4,
                                           shuffle=True)

val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                         batch_size=batch_size,
                                         num_workers=4)
print(f"Created data loaders! Batch size: {batch_size}")
print(f"Number of batches in train_loader: {len(train_loader)}")
print(f"Number of batches in val_loader: {len(val_loader)}")
print("--------------------------------------------------------------------------")

--------------------------------------------------------------------------
Create dataset...
Created dataset! Number of samples: 70150
--------------------------------------------------------------------------
Split dataset...
Splitted dataset! Number of training samples: 35075, number of validation samples: 35075
--------------------------------------------------------------------------
Create data loaders...
Created data loaders! Batch size: 256
Number of batches in train_loader: 138
Number of batches in val_loader: 138
--------------------------------------------------------------------------


<h3>Model building and training</h3>

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [5]:
print("--------------------------------------------------------------------------")
print("Create model...")
model = SiameseNetwork().to(device)
print("Created model!")
print(f'Model is on cuda: {next(model.parameters()).is_cuda}')
#print(model)
summary(model, [(3, 512, 512),(3, 512, 512),(3, 512, 512)], batch_size=batch_size)
print("--------------------------------------------------------------------------")

--------------------------------------------------------------------------
Create model...
Created model!
Model is on cuda: True
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1       [256, 128, 510, 510]           3,584
         MaxPool2d-2       [256, 128, 255, 255]               0
             PReLU-3       [256, 128, 255, 255]             128
            Conv2d-4        [256, 64, 253, 253]          73,792
         MaxPool2d-5        [256, 64, 126, 126]               0
             PReLU-6        [256, 64, 126, 126]              64
            Conv2d-7        [256, 32, 124, 124]          18,464
         MaxPool2d-8          [256, 32, 62, 62]               0
             PReLU-9          [256, 32, 62, 62]              32
           Conv2d-10          [256, 16, 60, 60]           4,624
        MaxPool2d-11          [256, 16, 30, 30]               0
            PReLU-12          [256, 16

C:\Users\samii\.conda\envs\applied-ml\lib\site-packages\torchsummary\torchsummary.py:93: RuntimeWarning: overflow encountered in long_scalars
  total_output += np.prod(summary[layer]["output_shape"])


In [6]:
print("--------------------------------------------------------------------------")
print("Create tensorboard_writer...")
tensorboard_writer = mytensorboard.MySummaryWriter(numb_batches=len(train_loader), batch_size=batch_size, experiment_name="SiameseNetwork")
print("Created tensorboard_writer!")
print("--------------------------------------------------------------------------")

--------------------------------------------------------------------------
Create tensorboard_writer...
nonempty logpath
logpath:  ..\logs\SiameseNetwork\run_1(2)
tensorboard is up: http://localhost:6006
Created tensorboard_writer!
--------------------------------------------------------------------------


In [7]:
print("--------------------------------------------------------------------------")
print("Create trainer...")
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
trainer = SiameseNetworkTrainer(model=model, 
                                train_loader=train_loader,
                                valid_loader=val_loader, 
                                test_loader=val_loader,
                                optimizer=optimizer, 
                                tensorboard_writer=tensorboard_writer)
print("Created trainer!")
print("--------------------------------------------------------------------------")

--------------------------------------------------------------------------
Create trainer...
Created trainer!
--------------------------------------------------------------------------


<h4>Tensorboard</h4>

In [8]:
# dataiter = iter(train_loader)
# images, labels = dataiter.next()

# anchors = images[0]
# positives = images[1]
# negatives = images[2]

# # write graph of model to tensorboard
# tensorboard_writer.add_graph(model, images)


# # write sample images to tensorboard


# anchors_grid = torchvision.utils.make_grid(anchors, nrow=batch_size)
# tensorboard_writer.add_image("anchor sample", anchors_grid)

# positives_grid = torchvision.utils.make_grid(positives, nrow=batch_size)
# tensorboard_writer.add_image("positives sample", positives_grid)

# negatives_grid = torchvision.utils.make_grid(negatives, nrow=batch_size)
# tensorboard_writer.add_image("negatives sample", negatives_grid)

# total_grid = torchvision.utils.make_grid([anchors_grid, positives_grid, negatives_grid], nrow=1)
# tensorboard_writer.add_image("sample", total_grid)

# fig = img_util.plot_classes_preds_face_recognition(anchors, labels, ["1234", "1234", "1234", "1234"])
# tensorboard_writer.add_figure("predictions vs. actuals", fig)

# time.sleep(13)

# # Deleting image variables to free RAM
# anchors_grid = None
# positives_grid = None
# negatives_grid = None
# total_grid = None
# fig = None

<h4>Training</h4>

In [ ]:
print("start training")

trainer.train(epochs=3)

start training


<h3>Model evaluation</h3>